# Shoreline Monitor

Notebook environment to migrate netcdf files to CF compliant zarr

In [31]:
# Optional; code formatter, installed as jupyter lab extension
#%load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

### Configure OS independent paths

In [32]:
# Import standard packages
import os
import pathlib

import sys
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import math

# Make root directories importable by appending root to path
cwd = pathlib.Path().resolve()
sys.path.append(os.path.dirname(cwd))
sys.path.append(r'P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\coclicodata')

# Get root paths
home = pathlib.Path().home()
root = home.root

# Import custom functionality
from etl import p_drive
from etl.CF_compliancy_checker import check_compliancy, save_compliancy

# Define (local and) remote drives
gca_data_dir = p_drive.joinpath("1000545-054-globalbeaches", "15_GlobalCoastalAtlas", "datasets")

# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    home.joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"Anaconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml"
    )
)

<IPython.core.display.Javascript object>

In [33]:
# Project paths & files (manual input)
dataset_dir = gca_data_dir.joinpath("01_Shorelinemonitor_annual")
dataset_dir_shorelinemonitor = dataset_dir.joinpath("shorelinemonitor_annual.nc")
dataset_out_file = "ShorelineMonitor"
CF_dir = gca_data_dir.joinpath(r"CF")  # directory to save output CF check files

<IPython.core.display.Javascript object>

In [34]:
# write csv to netcdf

# # Load data from CSV file into a pandas dataframe
# csv_dir = r'P:\1000545-054-globalbeaches\03_Global_shorelines_update_2020\Transect_Mapping\Version_16102018\CSV\ShorelineMonitor_1984_2021_v1.5_set1.csv'
# df = pd.read_csv(csv_dir)

# # Convert the pandas dataframe to an xarray dataset
# ds = xr.Dataset.from_dataframe(df)

# # Write the xarray dataset to a netCDF file
# ds.to_netcdf(dataset_dir)

<IPython.core.display.Javascript object>

### Check CF compliancy original NetCDF files

In [35]:
# open datasets
dataset_shorelinemonitor = xr.open_dataset(dataset_dir_shorelinemonitor)

# check original dataset
dataset_shorelinemonitor

<xarray.Dataset>
Dimensions:             (index: 1837203)
Coordinates:
  * index               (index) int64 0 1 2 3 ... 1837200 1837201 1837202
Data variables: (12/27)
    transect_id         (index) object ...
    country_id          (index) object ...
    continent           (index) object ...
    country_name        (index) object ...
    changerate          (index) float64 ...
    changerate_unc      (index) float64 ...
    ...                  ...
    End_lat             (index) float64 ...
    coastline_idint     (index) float64 ...
    no_sedcomp          (index) float64 ...
    low_detect_shlines  (index) float64 ...
    err_changerate      (index) float64 ...
    err_timespan        (index) float64 ...

<IPython.core.display.Javascript object>

In [36]:
%%capture cap --no-stderr
# check original CF compliancy

check_compliancy(testfile=dataset_dir_shorelinemonitor, 
                 working_dir=CF_dir
                 )


<IPython.core.display.Javascript object>

In [37]:
# save original CF compliancy
save_compliancy(cap, testfile=dataset_dir_shorelinemonitor, working_dir=CF_dir)

{'P:\\1000545-054-globalbeaches\\15_GlobalCoastalAtlas\\datasets\\01_Shorelinemonitor_annual\\shorelinemonitor_annual.nc': {'warnings': '38', 'errors': '8'}}


<IPython.core.display.Javascript object>

### Make CF compliant alterations to the NetCDF files (dataset dependent)

In [38]:
import json

# NetCDF attribute alterations
f_global = open(r'P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\datasets\01_Shorelinemonitor_annual\metadata_shorelinemonitor.json')
meta_global = json.load(f_global)

for attr_name, attr_val in meta_global.items():
    if attr_name == 'PROVIDERS':
        attr_val = json.dumps(attr_val)
    dataset_shorelinemonitor.attrs[attr_name] = attr_val

dataset_shorelinemonitor.attrs['Conventions'] = "CF-1.8"

<IPython.core.display.Javascript object>

In [39]:
# drop time xarray

#dataset_shorelinemonitor = dataset_shorelinemonitor.drop_vars('dt')
date_range = pd.date_range(start = '1984-01-01', end = '2021-01-01', freq = 'AS')
dataset_shorelinemonitor['time_step'] = xr.DataArray(date_range, dims = ['time']) #TODO: altered time_range into time_step


<IPython.core.display.Javascript object>

In [40]:
#combine start and end coordinates into a transect
from shapely.geometry import LineString

start_lons = dataset_shorelinemonitor['Start_lon'].values
start_lats = dataset_shorelinemonitor['Start_lat'].values
end_lons = dataset_shorelinemonitor['End_lon'].values
end_lats = dataset_shorelinemonitor['End_lat'].values
coords = zip(zip(start_lons,start_lats), zip(end_lons,end_lats))
dataset_shorelinemonitor['transect_geom'] = (['index'], [str(LineString(line)) for line in coords])
dataset_shorelinemonitor['transect_geom'].attrs['long_name'] = 'Transect Geometry'

<IPython.core.display.Javascript object>

In [41]:
def create_shorelinepos(time_range, positions, timesteps, intercept):
    l = np.full(len(time_range), np.nan)
    l[timesteps] = positions - intercept
    return l

new_var1 = xr.DataArray(
    np.array([create_shorelinepos(dataset_shorelinemonitor['time_step'].values, 
                                  np.array(json.loads(dataset_shorelinemonitor['dist'].values[i])), 
                                  np.array(json.loads(dataset_shorelinemonitor['dt'].values[i])).astype(int),
                                  dataset_shorelinemonitor['intercept'].values[i]) for i in range(len(dataset_shorelinemonitor['dist'].values))]),
    dims=['index', 'time_step'])


<IPython.core.display.Javascript object>

In [42]:
import gc

gc.collect()

261111

<IPython.core.display.Javascript object>

In [43]:
def combine_outliers(positions, outl_1, outl_2):
    l = np.zeros(len(positions))
    not_na = np.where(~np.isnan(positions))[0]
    outl_1, outl_2 = np.array(json.loads(outl_1)), json.loads(outl_2)
    outliers = sorted(list(outl_1) + [int(float(m) + sum(1 for i, n in enumerate(outl_1) if int(m) >= (n - i))) for m in outl_2])
    l[not_na[outliers]] = 1

    return l

new_var2 = xr.apply_ufunc(
    combine_outliers,
    new_var1,
    dataset_shorelinemonitor['outliers_1'],
    dataset_shorelinemonitor['outliers_2'],
    input_core_dims=[['time_step'], [], []],
    output_core_dims=[['time_step']],
    vectorize=True,
    dask='parallelized',
    output_dtypes=[np.float32]
)

<IPython.core.display.Javascript object>

In [44]:
delete_vars = ['dist', 'outliers_1', 'outliers_2']
for dv in delete_vars:
    dataset_shorelinemonitor = dataset_shorelinemonitor.drop(dv)

dataset_shorelinemonitor['dist'] = (['index', 'time'], new_var1.data)
dataset_shorelinemonitor['outliers'] = (['index', 'time'], new_var2.data)

<IPython.core.display.Javascript object>

In [45]:
gc.collect()

3784

<IPython.core.display.Javascript object>

In [46]:
dataset_shorelinemonitor['outliers'].values[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0.], dtype=float32)

<IPython.core.display.Javascript object>

In [47]:
dataset_shorelinemonitor

<xarray.Dataset>
Dimensions:             (index: 1837203, time: 38)
Coordinates:
  * index               (index) int64 0 1 2 3 ... 1837200 1837201 1837202
  * time                (time) datetime64[ns] 1984-01-01 ... 2021-01-01
Data variables: (12/28)
    transect_id         (index) object ...
    country_id          (index) object ...
    continent           (index) object ...
    country_name        (index) object ...
    changerate          (index) float64 ...
    changerate_unc      (index) float64 ...
    ...                  ...
    err_changerate      (index) float64 ...
    err_timespan        (index) float64 ...
    time_step           (time) datetime64[ns] 1984-01-01 ... 2021-01-01
    transect_geom       (index) <U92 'LINESTRING (-74.3863095545 -50.37765894...
    dist                (index, time) float64 nan nan nan nan ... nan 370.5 nan
    outliers            (index, time) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
Attributes: (12/21)
    TITLE:               The Shoreline Monitor
    TITLE_ABBREVIATION:  shore_mon
    DESCRIPTION:         The Shoreline Monitor is a dataset that incorporates...
    SHORT_DESCRIPTION:   The Shoreline Monitor is a global dataset that provi...
    INSTITUTION:         Deltares
    PROVIDERS:           {"name": "Deltares", "url": "www.deltares.nl", "role...
    ...                  ...
    DOI:                 https://doi.org/10.1038/s41598-018-24630-6
    LONG_NAME:           THE_SHORELINE_MONITOR
    UNITS:               
    COMMENT:             
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [48]:
# NetCDF variable and dimension alterations

# rename or swap dimension names, the latter in case the name already exists as coordinate
dataset_shorelinemonitor = dataset_shorelinemonitor.rename_dims({"index": "stations"})

f_vars = open(r'P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\datasets\01_Shorelinemonitor_annual\vars_shorelinemonitor.json')
meta_vars = json.load(f_vars)

delete_vars = ['Intersect_lon', 'Intersect_lat', 'End_lon', 'End_lat', 'dt']
for dv in delete_vars:
    dataset_shorelinemonitor = dataset_shorelinemonitor.drop(dv)
    
for var_name, var_dict in meta_vars.items():
    dataset_shorelinemonitor = dataset_shorelinemonitor.rename_vars({var_name : var_dict['name']})
    for key, value in var_dict.items():
        if key != 'name':
            dataset_shorelinemonitor[var_dict['name']].attrs[key] = value

# change dtypes
object_vars = ['transect_id', 'country_id', 'continent', 'country', 'transect_geom']
for ov in object_vars:
    dataset_shorelinemonitor[ov] = dataset_shorelinemonitor[ov].astype('S')

all_vars = list(dataset_shorelinemonitor.keys())
data_vars = ['changerate', 'changerate_unc', 'sandy', 'sp', 'outliers']
# set some data variables to coordinates to avoid duplication of dimensions in later stage
dataset_shorelinemonitor = dataset_shorelinemonitor.set_coords([v for v in all_vars if v not in data_vars])

# drop index xarray
dataset_shorelinemonitor = dataset_shorelinemonitor.drop('index')

<IPython.core.display.Javascript object>

In [49]:
dataset_shorelinemonitor

<xarray.Dataset>
Dimensions:             (stations: 1837203, time: 38)
Coordinates: (12/19)
    transect_id         (stations) |S15 b'BOX_028_183_0' ... b'BOX_211_067_153'
    country_id          (stations) |S7 b'CHL' b'CHL' b'CHL' ... b'RUS' b'RUS'
    continent           (stations) |S23 b'South America' ... b'Europe'
    country             (stations) |S40 b'Chile' b'Chile' ... b'Russia'
    no_shorelines       (stations) float64 ...
    rmse                (stations) float64 ...
    ...                  ...
    low_detect_shlined  (stations) float64 ...
    err_changerate      (stations) float64 ...
    err_timespan        (stations) float64 ...
  * time                (time) datetime64[ns] 1984-01-01 ... 2021-01-01
    time_step           (time) datetime64[ns] 1984-01-01 ... 2021-01-01
    transect_geom       (stations) |S92 b'LINESTRING (-74.3863095545 -50.3776...
Dimensions without coordinates: stations
Data variables:
    changerate          (stations) float64 ...
    changerate_unc      (stations) float64 ...
    sandy               (stations) bool ...
    sp                  (stations, time) float64 nan nan nan ... nan 370.5 nan
    outliers            (stations, time) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
Attributes: (12/21)
    TITLE:               The Shoreline Monitor
    TITLE_ABBREVIATION:  shore_mon
    DESCRIPTION:         The Shoreline Monitor is a dataset that incorporates...
    SHORT_DESCRIPTION:   The Shoreline Monitor is a global dataset that provi...
    INSTITUTION:         Deltares
    PROVIDERS:           {"name": "Deltares", "url": "www.deltares.nl", "role...
    ...                  ...
    DOI:                 https://doi.org/10.1038/s41598-018-24630-6
    LONG_NAME:           THE_SHORELINE_MONITOR
    UNITS:               
    COMMENT:             
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [50]:
dataset_shorelinemonitor = dataset_shorelinemonitor.drop('time_step')

<IPython.core.display.Javascript object>

In [51]:
#dataset_shorelinemonitor = dataset_shorelinemonitor.rename_vars({'time_step' : 'time'})
rename = {"long_name": "Time", "standard_name": "time", "units": "yr"}

dataset_shorelinemonitor['time'].attrs['long_name'] = rename['long_name']
dataset_shorelinemonitor['time'].attrs['standard_name'] = rename['standard_name']
dataset_shorelinemonitor['time'].attrs['units'] = rename['units']

<IPython.core.display.Javascript object>

In [52]:
# check the xarray dataset, best practice is to have as many as possible bold dimensions (dimension == coordinate name).
# in this way, the Front-End can access the variable directly without having to index the variable first
# dataset["scenarios"]
#dataset_shorelinemonitor = dataset_shorelinemonitor.rename_dims({'time_step': 'ntime'})
dataset_shorelinemonitor

<xarray.Dataset>
Dimensions:             (stations: 1837203, time: 38)
Coordinates: (12/18)
    transect_id         (stations) |S15 b'BOX_028_183_0' ... b'BOX_211_067_153'
    country_id          (stations) |S7 b'CHL' b'CHL' b'CHL' ... b'RUS' b'RUS'
    continent           (stations) |S23 b'South America' ... b'Europe'
    country             (stations) |S40 b'Chile' b'Chile' ... b'Russia'
    no_shorelines       (stations) float64 ...
    rmse                (stations) float64 ...
    ...                  ...
    no_sedcomp          (stations) float64 ...
    low_detect_shlined  (stations) float64 ...
    err_changerate      (stations) float64 ...
    err_timespan        (stations) float64 ...
  * time                (time) datetime64[ns] 1984-01-01 ... 2021-01-01
    transect_geom       (stations) |S92 b'LINESTRING (-74.3863095545 -50.3776...
Dimensions without coordinates: stations
Data variables:
    changerate          (stations) float64 ...
    changerate_unc      (stations) float64 ...
    sandy               (stations) bool ...
    sp                  (stations, time) float64 nan nan nan ... nan 370.5 nan
    outliers            (stations, time) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
Attributes: (12/21)
    TITLE:               The Shoreline Monitor
    TITLE_ABBREVIATION:  shore_mon
    DESCRIPTION:         The Shoreline Monitor is a dataset that incorporates...
    SHORT_DESCRIPTION:   The Shoreline Monitor is a global dataset that provi...
    INSTITUTION:         Deltares
    PROVIDERS:           {"name": "Deltares", "url": "www.deltares.nl", "role...
    ...                  ...
    DOI:                 https://doi.org/10.1038/s41598-018-24630-6
    LONG_NAME:           THE_SHORELINE_MONITOR
    UNITS:               
    COMMENT:             
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [53]:
# save new .nc files
dataset_shorelinemonitor['time'].attrs.pop('units', None)
dataset_shorelinemonitor.to_netcdf(path=str(dataset_dir_shorelinemonitor).replace(".nc", "_CF.nc"))

<IPython.core.display.Javascript object>

### Check CF compliancy altered NetCDF files

In [54]:
%%capture cap --no-stderr
# check altered CF compliancy

check_compliancy(testfile=str(dataset_dir_shorelinemonitor).replace(".nc", "_CF.nc"), working_dir=CF_dir)

<IPython.core.display.Javascript object>

In [55]:
# save altered CF compliancy
save_compliancy(
    cap,
    testfile=str(dataset_dir_shorelinemonitor).replace(".nc", "_CF.nc"),
    working_dir=CF_dir,
)

{'P:\\1000545-054-globalbeaches\\15_GlobalCoastalAtlas\\datasets\\01_Shorelinemonitor_annual\\shorelinemonitor_annual_CF.nc': {'warnings': '4', 'errors': '0'}}


<IPython.core.display.Javascript object>

### write data to Zarr files

In [56]:
dataset_shorelinemonitor['outliers'].values[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0.], dtype=float32)

<IPython.core.display.Javascript object>

In [57]:
# export to zarr in write mode (to overwrite if exists)
dataset_shorelinemonitor.to_zarr(dataset_dir.joinpath("%s.zarr" % dataset_out_file), mode="w")

<IPython.core.display.Javascript object>

In [58]:
test = xr.open_zarr(dataset_dir.joinpath("%s.zarr" % dataset_out_file))

<IPython.core.display.Javascript object>

In [59]:
test

<xarray.Dataset>
Dimensions:             (stations: 1837203, time: 38)
Coordinates: (12/18)
    coastline_idint     (stations) float64 dask.array<chunksize=(57413,), meta=np.ndarray>
    continent           (stations) |S23 dask.array<chunksize=(28707,), meta=np.ndarray>
    country             (stations) |S40 dask.array<chunksize=(28707,), meta=np.ndarray>
    country_id          (stations) |S7 dask.array<chunksize=(114826,), meta=np.ndarray>
    err_changerate      (stations) float64 dask.array<chunksize=(57413,), meta=np.ndarray>
    err_timespan        (stations) float64 dask.array<chunksize=(57413,), meta=np.ndarray>
    ...                  ...
    no_shorelines       (stations) float64 dask.array<chunksize=(57413,), meta=np.ndarray>
    rmse                (stations) float64 dask.array<chunksize=(57413,), meta=np.ndarray>
  * time                (time) datetime64[ns] 1984-01-01 ... 2021-01-01
    timespan            (stations) float64 dask.array<chunksize=(57413,), meta=np.ndarray>
    transect_geom       (stations) |S92 dask.array<chunksize=(14354,), meta=np.ndarray>
    transect_id         (stations) |S15 dask.array<chunksize=(57413,), meta=np.ndarray>
Dimensions without coordinates: stations
Data variables:
    changerate          (stations) float64 dask.array<chunksize=(57413,), meta=np.ndarray>
    changerate_unc      (stations) float64 dask.array<chunksize=(57413,), meta=np.ndarray>
    outliers            (stations, time) float32 dask.array<chunksize=(114826, 3), meta=np.ndarray>
    sandy               (stations) int8 dask.array<chunksize=(459301,), meta=np.ndarray>
    sp                  (stations, time) float64 dask.array<chunksize=(57413, 3), meta=np.ndarray>
Attributes: (12/21)
    AUTHOR:              Luijendijk, A., Hagenaars, G., Ranasinghe, R., Baart...
    CITATION:            Luijendijk, A., Hagenaars, G., Ranasinghe, R. et al....
    COMMENT:             
    CRS:                 EPSG:4326
    Conventions:         CF-1.8
    DESCRIPTION:         The Shoreline Monitor is a dataset that incorporates...
    ...                  ...
    SPATIAL_EXTENT:      [-180, -90, 180, 90]
    TAGS:                ['civil engineering', 'geomorphology', 'shoreline dy...
    TEMPORAL_EXTENT:     ['1984-01-01T00:00:00Z', '2021-01-01T00:00:00Z']
    TITLE:               The Shoreline Monitor
    TITLE_ABBREVIATION:  shore_mon
    UNITS:

<IPython.core.display.Javascript object>